In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Cargamos el archivo csv
df=pd.read_csv('Euskadi_Limpio.csv')
#Eliminamos la columna unnamed
df= df.drop(['Unnamed: 0'], axis=1)
df.head(20)

,last_scraped,source,description,neighborhood_overview,host_name,host_since,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2024-09-30,city scrape,"Amazing modern, clean & fully equiped flat of...",Gros is Donosti's trendy neighborhood. The apa...,Mikel,2011-05-31,within an hour,t,Unknown,"['email', 'phone']",...,4.77,4.97,4.97,4.94,4.75,1.0,1.0,0.0,0.0,1.10
1,2024-09-29,city scrape,Beach front apartment close to San Juan de Gaz...,No overview available,Maitane,2011-06-12,within an hour,t,La Latina,"['email', 'phone']",...,4.91,4.51,4.63,4.70,4.70,1.0,1.0,0.0,0.0,0.50
2,2024-09-30,city scrape,The apartment is located in the mezzanine of a...,In this neighborhood with a strong personality...,Emy,2012-01-10,within an hour,f,Unknown,"['email', 'phone']",...,4.80,4.67,4.90,5.00,4.20,7.0,6.0,1.0,0.0,0.13
3,2024-09-30,city scrape,No description available,The apartment is located in the Romantic Area ...,Emy,2012-01-10,within an hour,f,Unknown,"['email', 'phone']",...,4.83,4.83,4.92,4.92,4.58,7.0,6.0,1.0,0.0,0.09
4,2024-09-30,city scrape,No description available,The apartment is located in the area known as ...,Emy,2012-01-10,within an hour,f,Unknown,"['email', 'phone']",...,4.82,4.75,4.71,4.81,4.75,7.0,6.0,1.0,0.0,0.11
5,2024-09-30,city scrape,Belle View is an elegant apartament located on...,The apartment is located in the area known as ...,Emy,2012-01-10,within an hour,f,Unknown,"['email', 'phone']",...,4.91,4.94,4.94,4.97,4.88,7.0,6.0,1.0,0.0,0.26
6,2024-09-30,city scrape,Centre Ville is an apartament located on the t...,The apartment is located in the area known as ...,Emy,2012-01-10,within an hour,f,Unknown,"['email', 'phone']",...,4.69,4.93,4.83,4.85,4.58,7.0,6.0,1.0,0.0,0.74
7,2024-09-29,city scrape,No description available,No overview available,MartÃ­n,2012-02-07,within a few hours,f,Unknown,"['email', 'phone']",...,4.67,4.80,4.90,4.33,4.33,2.0,2.0,0.0,0.0,0.22
8,2024-09-29,city scrape,"The room consists of a 1.50 bed, dresser, clos...","the area is quiet , except if there is a socce...",Itziar,2012-02-11,within a few hours,t,Unknown,"['email', 'phone']",...,4.81,4.88,4.88,4.85,4.77,1.0,0.0,1.0,0.0,0.56
9,2024-09-29,city scrape,Fully equipped luxury villa 10 minutes from Sa...,No overview available,Joxeanjel,2012-02-07,within a few hours,f,Unknown,"['email', 'phone']",...,5.00,5.00,5.00,5.00,4.94,1.0,1.0,0.0,0.0,0.34


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6891 entries, 0 to 6890
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  6891 non-null   object 
 1   source                                        6891 non-null   object 
 2   description                                   6891 non-null   object 
 3   neighborhood_overview                         6891 non-null   object 
 4   host_name                                     6891 non-null   object 
 5   host_since                                    6891 non-null   object 
 6   host_response_time                            6891 non-null   object 
 7   host_is_superhost                             6891 non-null   object 
 8   host_neighbourhood                            6891 non-null   object 
 9   host_verifications                            6891 non-null   o

In [5]:
#Verificar los valores sin repetirse de una columna
unico1 = np.unique(df['room_type'])
unico1

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [6]:
#Verificar los valores sin repetirse de una columna
unico2 = np.unique(df['bathrooms_text'])
unico2

array(['0 baths', '0 shared baths', '1 bath', '1 private bath',
       '1 shared bath', '1.5 baths', '1.5 shared baths', '11 baths',
       '12.5 baths', '13.5 baths', '2 baths', '2 shared baths',
       '2.5 baths', '2.5 shared baths', '24 shared baths', '3 baths',
       '3 shared baths', '3.5 baths', '3.5 shared baths', '4 baths',
       '4 shared baths', '4.5 baths', '4.5 shared baths', '5 baths',
       '5 shared baths', '5.5 baths', '5.5 shared baths', '6 baths',
       '6 shared baths', '6.5 baths', '6.5 shared baths', '7 baths',
       '7 shared baths', '7.5 baths', '7.5 shared baths', '8 baths',
       '8 shared baths', '8.5 baths', '9 baths', '9.5 baths', 'Half-bath',
       'Private half-bath', 'Shared half-bath'], dtype=object)

In [7]:
#Convierto una variable a dicotómica
df['room_type']= df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "No Entire home/apt")
df['room_type']

0          Entire home/apt
1          Entire home/apt
2          Entire home/apt
3          Entire home/apt
4          Entire home/apt
               ...        
6886       Entire home/apt
6887    No Entire home/apt
6888       Entire home/apt
6889       Entire home/apt
6890       Entire home/apt
Name: room_type, Length: 6891, dtype: object

In [8]:
# Extraer el número de baños como flotante
df["bathrooms_num"] = df["bathrooms_text"].str.extract(r'(\d+\.?\d*)').astype(float)

# Crear la variable dicotómica de manera automática
df["bathrooms_text"] = np.where(df["bathrooms_num"] > 1, ">1 baño", "≤1 baño")

# Verificar los cambios
df["bathrooms_text"].value_counts()


bathrooms_text
≤1 baño    4166
>1 baño    2725
Name: count, dtype: int64

In [9]:
#Convierto una variable a dicotómica
df['host_is_superhost']= df['host_is_superhost'].replace(["0"], "f")
df['host_is_superhost']

0       t
1       t
2       f
3       f
4       f
       ..
6886    f
6887    f
6888    t
6889    f
6890    f
Name: host_is_superhost, Length: 6891, dtype: object

In [10]:
#Seleccion de variables dependientes
Var_Dep1 = df["host_is_superhost"].unique()
print(Var_Dep1)
Var_Dep2 = df["instant_bookable"].unique()
print(Var_Dep2)
Var_Dep3 = df["host_identity_verified"].unique()
print(Var_Dep3)
Var_Dep4 = df["room_type"].unique()
print(Var_Dep4)
Var_Dep5 = df["bathrooms_text"].unique()
print(Var_Dep5)

['t' 'f']
['t' 'f']
['t' 'f']
['Entire home/apt' 'No Entire home/apt']
['≤1 baño' '>1 baño']


**Primer modelo de regresion logica**

In [11]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep= df[['price']]
Var_Dep1= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep
y= Var_Dep1

In [12]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [13]:
#Se escalan todos los datos
escalar = StandardScaler()

In [14]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [15]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [16]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [17]:
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [18]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confunsion 1:')
print(matriz)

Matriz de Confunsion 1:
[[1309    0]
 [ 759    0]]


In [19]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo 1:')
print(precision)

Precision del modelo 1:
0.6329787234042553


In [20]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo 1:')
print(exactitud)

Exactitud del modelo 1:
0.6329787234042553


In [21]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo 1:')
print(sensibilidad)

Sensibilidad del modelo 1:
0.0


**Segundo modelo umu**

In [22]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep2= df[['review_scores_rating', 'review_scores_communication', 'review_scores_checkin']]
Var_Dep2= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep2
y= Var_Dep2

In [23]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [24]:
#Realizamos una prediccion
y_pred2 = algoritmo.predict(X_test)
y_pred2

array(['t', 'f', 'f', ..., 'f', 't', 'f'], dtype=object)

In [25]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred2)
print('Matriz de Confunsion 2:')
print(matriz)

Matriz de Confunsion 2:
[[991 275]
 [531 271]]


In [26]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred2, average="binary", pos_label="t")
print('Precision del modelo 2:')
print(precision)

Precision del modelo 2:
0.49633699633699635


In [27]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred2)
print('Exactitud del modelo 2:')
print(exactitud)

Exactitud del modelo 2:
0.6102514506769826


In [28]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad del modelo 2:')
print(sensibilidad)

Sensibilidad del modelo 2:
0.782780410742496


**Tercer modelo de regresion logistica**

In [31]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep3= df[['review_scores_rating', 'price', 'host_acceptance_rate']]
Var_Dep3= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep3
y= Var_Dep3

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)


LogisticRegression()

In [32]:

#Realizamos una prediccion
y_pred3 = algoritmo.predict(X_test)
y_pred3

array(['t', 't', 'f', ..., 't', 't', 't'], dtype=object)

In [33]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred3)
print('Matriz de Confunsion 3:')
print(matriz)

Matriz de Confunsion 3:
[[711 368]
 [419 570]]


In [34]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred3, average="binary", pos_label="t")
print('Precision del modelo 3:')
print(precision)

Precision del modelo 3:
0.6076759061833689


In [35]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred3)
print('Exactitud del modelo 3:')
print(exactitud)

Exactitud del modelo 3:
0.6194390715667312


In [36]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad del modelo 3:')
print(sensibilidad)

Sensibilidad del modelo 3:
0.6589434661723819


**CUARTO MODELO DE REGRESION**

In [41]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep4= df[['number_of_reviews', 'price']]
Var_Dep4= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep4
y= Var_Dep4


In [42]:

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [43]:
#Realizamos una prediccion
y_pred4 = algoritmo.predict(X_test)
y_pred4

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [44]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred4)
print('Matriz de Confunsion 4:')
print(matriz)

Matriz de Confunsion 4:
[[803 298]
 [648 319]]


In [45]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred4, average="binary", pos_label="f")
print('Precision del modelo 4:')
print(precision)

Precision del modelo 4:
0.5534114403859407


In [46]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred4)
print('Exactitud del modelo 4:')
print(exactitud)

Exactitud del modelo 4:
0.5425531914893617


In [47]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred4, average="binary", pos_label="t")
print('Sensibilidad del modelo 4:')
print(sensibilidad)

Sensibilidad del modelo 4:
0.3298862461220269


**Quinto modelo de regresion logistica**

In [48]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep5= df[['number_of_reviews', 'price', 'host_total_listings_count']]
Var_Dep5= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep5
y= Var_Dep5

In [49]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [50]:
#Realizamos una prediccion
y_pred5 = algoritmo.predict(X_test)
y_pred5

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [51]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred5)
print('Matriz de Confunsion 5:')
print(matriz)

Matriz de Confunsion 5:
[[   0   45]
 [   0 2023]]


In [52]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred5, average="binary", pos_label="t")
print('Precision del modelo 5:')
print(precision)

Precision del modelo 5:
0.9782398452611218


In [53]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred5)
print('Exactitud del modelo 5:')
print(exactitud)

Exactitud del modelo 5:
0.9782398452611218


In [54]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred5, average="binary", pos_label="f")
print('Sensibilidad del modelo 5:')
print(sensibilidad)

Sensibilidad del modelo 5:
0.0


**SEXTO MODELO DE REGRESION LOGISTICA**

In [55]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep6= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep6= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep6
y= Var_Dep6

In [56]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [57]:
#Realizamos una prediccion
y_pred6 = algoritmo.predict(X_test)
y_pred6

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [58]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred6)
print('Matriz de Confunsion 6:')
print(matriz)

Matriz de Confunsion 6:
[[   0   50]
 [   0 2018]]


In [59]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred6, average="binary", pos_label="t")
print('Precision del modelo 6:')
print(precision)

Precision del modelo 6:
0.9758220502901354


In [60]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred6)
print('Exactitud del modelo 6:')
print(exactitud)

Exactitud del modelo 6:
0.9758220502901354


In [61]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred6, average="binary", pos_label="f")
print('Sensibilidad del modelo 6:')
print(sensibilidad)

Sensibilidad del modelo 6:
0.0


**SEPTIMO MODELO DE REGRESION**

In [62]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep7= df[['accommodates', 'host_acceptance_rate', 'price']]
Var_Dep7= df['room_type']

#Redefinimos las variables
X= Vars_Indep7
y= Var_Dep7

In [63]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [64]:
#Realizamos una prediccion
y_pred7 = algoritmo.predict(X_test)
y_pred7

array(['Entire home/apt', 'Entire home/apt', 'No Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      dtype=object)

In [65]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred7)
print('Matriz de Confunsion 7:')
print(matriz)

Matriz de Confunsion 7:
[[1357  134]
 [ 159  418]]


In [66]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred7, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 7:')
print(precision)

Precision del modelo 7:
0.7572463768115942


In [67]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred7)
print('Exactitud del modelo 7:')
print(exactitud)

Exactitud del modelo 7:
0.8583172147001934


In [68]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred7, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 7:')
print(sensibilidad)

Sensibilidad del modelo 7:
0.9101274312541918


**OCTAVO MODELO**

In [69]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep8= df[['number_of_reviews', 'bedrooms']]
Var_Dep8= df['room_type']

#Redefinimos las variables
X= Vars_Indep8
y= Var_Dep8

In [70]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [71]:
#Realizamos una prediccion
y_pred8 = algoritmo.predict(X_test)
y_pred8

array(['Entire home/apt', 'Entire home/apt', 'No Entire home/apt', ...,
       'Entire home/apt', 'No Entire home/apt', 'Entire home/apt'],
      dtype=object)

In [72]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred8)
print('Matriz de Confunsion 8:')
print(matriz)

Matriz de Confunsion 8:
[[1202  252]
 [ 209  405]]


In [73]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred8, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 8:')
print(precision)

Precision del modelo 8:
0.6164383561643836


In [74]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred8)
print('Exactitud del modelo 8:')
print(exactitud)

Exactitud del modelo 8:
0.7770793036750484


In [75]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred8, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 8:')
print(sensibilidad)

Sensibilidad del modelo 8:
0.8266850068775791


**NOVENO MODELO**

In [76]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep9= df[['number_of_reviews', 'price', 'accommodates']]
Var_Dep9= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep9
y= Var_Dep9

In [77]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [78]:
#Realizamos una prediccion
y_pred9 = algoritmo.predict(X_test)
y_pred9

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '≤1 baño', '>1 baño',
       '>1 baño'], dtype=object)

In [79]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred9)
print('Matriz de Confunsion 9:')
print(matriz)

Matriz de Confunsion 9:
[[ 440  358]
 [ 147 1123]]


In [80]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred9, average="binary", pos_label=">1 baño")
print('Precision del modelo 9:')
print(precision)

Precision del modelo 9:
0.7495741056218058


In [81]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred9)
print('Exactitud del modelo 9:')
print(exactitud)

Exactitud del modelo 9:
0.7558027079303675


In [82]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred9, average="binary", pos_label="≤1 baño")
print('Sensibilidad del modelo 9:')
print(sensibilidad)

Sensibilidad del modelo 9:
0.8842519685039371


**DECIMO MODELO**

In [83]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep10= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep10= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep10
y= Var_Dep10

In [84]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [85]:
#Realizamos una prediccion
y_pred10 = algoritmo.predict(X_test)
y_pred10

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], dtype=object)

In [86]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred10)
print('Matriz de Confunsion 10:')
print(matriz)

Matriz de Confunsion 10:
[[   0  815]
 [   0 1253]]


In [87]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred10, average="binary", pos_label="≤1 baño")
print('Precision del modelo 10:')
print(precision)

Precision del modelo 10:
0.6058994197292069


In [88]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred10)
print('Exactitud del modelo 10:')
print(exactitud)

Exactitud del modelo 10:
0.6058994197292069


In [89]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred10, average="binary", pos_label=">1 baño")
print('Sensibilidad del modelo 10:')
print(sensibilidad)

Sensibilidad del modelo 10:
0.0
